# U.S. EPA Food Commodity Intake Database (FCID)
## [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [ ]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [ ]:
SELECT * FROM FCID_Description LIMIT 10;

In [ ]:
SELECT * FROM Recipes LIMIT 10;

In [11]:
SELECT * FROM Intake LIMIT 10;

## 1)  Apresentando uma Receita

* Liste os ingredientes da receita de código `27111300` - Mexican style beef stew, no potatoes, tomato-based sauce (mixture).
* Não devem aparecer as modificações da receita.
* Mostre apenas o código da receita, o código de cada ingrediente, sua ordem e a participação no peso.

In [37]:
SELECT Food_Code, FCID_Code, Ingredient_Num, Commodity_Weight
    FROM Recipes
    WHERE
        Food_Code = '27111300'
        AND Mod_Code = 0
    ORDER BY Ingredient_Num;

## 2) Receita com Nomes de Ingredientes
* Aprimore a solução (1) para apresentar o nome dos ingredientes junto com seus códigos.

In [38]:
SELECT Food_Code, FCID_Desc, Recipes.FCID_Code, Ingredient_Num, Commodity_Weight
    FROM Recipes, FCID_Description
    WHERE
        Food_Code = '27111300'
        AND Mod_Code = 0
        AND FCID_Description.FCID_Code = Recipes.FCID_Code;

## 3) Grupos Alimentares da Receita
* A partir da receita escolhida em (1), apresente os grupos alimentares dos ingredientes contidos na receita.
* Cada grupo alimentar só deve aparecer uma vez no resultado.
* Para se obter o nome do grupo alimentar na tabela `Crop_Group` (não o subgrupo) devem ser considerados os registros em que a coluna `CGN` é igual à `CGL`.

In [30]:
SELECT Crop_Group.CGN, Crop_Group_Description
    FROM Recipes, FCID_Description, Crop_Group
    WHERE
        Recipes.Food_Code = '27111300'
        AND Recipes.Mod_Code = 0
        AND FCID_Description.FCID_Code = Recipes.FCID_Code
        AND Crop_Group.CGN = FCID_Description.CGN
        AND Crop_Group.CGN = Crop_Group.CGL
    GROUP BY Crop_Group.CGN, Crop_Group_Description;

## 4) Participação nas Receitas
* Liste o nome de cada um dos produtos alimentares seguido do número de receitas em que ele participa.
* A lista deve ser apresentada em ordem decrescente de número de participação em receitas.

In [23]:
SELECT FCID_Desc, COUNT(*) AS participacao_em_receitas
    FROM FCID_Description, Recipes
    WHERE FCID_Description.FCID_Code = Recipes.FCID_Code
    GROUP BY FCID_Desc
    ORDER BY participacao_em_receitas DESC;

## 5) Consumo Médio de Grupos de Alimentos
* Para cada Grupo de Alimentos (cada CGN diferente na tabela Crop_Group), apresente sua descrição e o consumo médio deste grupo de alimentos.
* O consumo é definido pelo campo `Intake` da tabela `Intake`.

In [21]:
SELECT Crop_Group.CGN, Crop_Group.Crop_Group_Description, AVG(Intake) AS consumo_medio
    FROM Crop_Group, Intake, FCID_Description
    WHERE
        Intake.FCID_Code = FCID_Description.FCID_Code 
        AND FCID_Description.CGN = Crop_Group.CGN
        AND Crop_Group.CGL = Crop_Group.CGN
    GROUP BY Crop_Group.CGN, Crop_Group.Crop_Group_Description;